In [1]:
import params as par
import source
import geopandas as gpd
import matplotlib.pyplot as plt
plt.style.use('default')

In [2]:
source.make_video.main()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


1669825035879_kochi_small/figures/Figure_0000.png
1669825035879_kochi_small/figures/Figure_0001.png
1669825035879_kochi_small/figures/Figure_0002.png
1669825035879_kochi_small/figures/Figure_0003.png
1669825035879_kochi_small/figures/Figure_0004.png
1669825035879_kochi_small/figures/Figure_0005.png
1669825035879_kochi_small/figures/Figure_0006.png
1669825035879_kochi_small/figures/Figure_0007.png
1669825035879_kochi_small/figures/Figure_0008.png
1669825035879_kochi_small/figures/Figure_0009.png
1669825035879_kochi_small/figures/Figure_0010.png
1669825035879_kochi_small/figures/Figure_0011.png
1669825035879_kochi_small/figures/Figure_0012.png
1669825035879_kochi_small/figures/Figure_0013.png
1669825035879_kochi_small/figures/Figure_0014.png
1669825035879_kochi_small/figures/Figure_0015.png
1669825035879_kochi_small/figures/Figure_0016.png
1669825035879_kochi_small/figures/Figure_0017.png
1669825035879_kochi_small/figures/Figure_0018.png
1669825035879_kochi_small/figures/Figure_0019.png


In [ ]:
#Verify AOS
aos = gpd.read_file(par.AOS_FILE).to_crs(par.WK_CRS)
print(aos.crs)
aos.explore()

In [ ]:
import osmnx as ox

aos_gdf = aos
area = {
        "north": aos_gdf.total_bounds[3],
        "south": aos_gdf.total_bounds[1],
        "east": aos_gdf.total_bounds[2],
        "west": aos_gdf.total_bounds[0],
    }

G = source.preprocess.createGraph(area=area, crs=par.PJ_CRS, ntype=par.OSM_NTYPE, plot=False)
n, e = ox.graph_to_gdfs(G)

In [ ]:
e[e['osmid'].islist()]

In [ ]:
e.drop_duplicates(subset=['u_original','v_original'],inplace=True)


In [ ]:
e.reset_index(inplace=True)
e.head()

In [ ]:
e.to_file('./edges.shp')

In [ ]:
fig,axs = plt.subplots(1,7,sharex=True,sharey=True,figsize=(30,10))
plt.tight_layout()
shi_pol.plot(ax=axs[0])
# shi_town.plot(ax=axs[1],column='Val_GovCod')
# census.plot(ax=axs[2],column='M_TOTPOP_H',cmap='jet')
buildings.plot(ax=axs[3])
shelters.plot(ax=axs[4],column='HNJ_SITEI',cmap='jet')
mesh.plot(ax=axs[5],column='MESH4_ID',cmap='jet')
# edges.plot(ax=axs[6],color='grey',lw=1)
# nodes.plot(ax=axs[6],markersize=3,color='k')


In [ ]:
# aos = gpd.read_file(f'./case/{case}/aos.geojson',driver='GeoJSON')
aos = shi_pol

In [ ]:
c = census.clip(aos)
# sh = shelters.clip(aos)
m = mesh.clip(aos)
e = edges.clip(aos)
n = nodes.clip(aos)
# t = shi_town.clip(aos)

In [ ]:
fig,axs = plt.subplots(5,1,sharex=True,sharey=True,figsize=(10,20))
plt.tight_layout()
c.plot(ax=axs[0], column='M_TOTPOP_H',cmap='jet')
# sh.plot(ax=axs[1], column='capacity_per',cmap='jet')
m.plot(ax=axs[2], column='MESH4_ID',cmap='jet')
e.plot(ax=axs[3], color='grey',lw=1)
n.plot(ax=axs[3], markersize=3)
# t.plot(ax=axs[4], column='Val_GovCod')

In [ ]:
c.to_file(f'./case/{case}/aos_census.geojson',driver='GeoJSON')
# sh.to_file(f'./case/{case}/aos_shelters.geojson',driver='GeoJSON')
m.to_file(f'./case/{case}/aos_mesh.geojson',driver='GeoJSON')
e.to_file(f'./case/{case}/aos_edges_esri.geojson',driver='GeoJSON')
n.to_file(f'./case/{case}/aos_nodes_esri.geojson',driver='GeoJSON')
# t.to_file(f'./case/{case}/aos_town.geojson',driver='GeoJSON')

In [ ]:
import geopandas as gpd
gdf = gpd.read_file('./case/kochi/aos_census.geojson', driver="GeoJSON")
gdf.iloc[:,:-1]